<a href="https://colab.research.google.com/github/FrancoDebord/Traduction_Fr_Fon/blob/master/avec_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
import io

uploaded = files.upload()




Saving francais-fongbe-revu_sans-doublons2.txt to francais-fongbe-revu_sans-doublons2.txt


In [0]:

  

import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt

import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense, Dropout, Bidirectional
from keras.models import Model
from keras.optimizers import RMSprop
from keras import regularizers
from keras import initializers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2, l1
import io
import pickle


lines = pd.read_csv(io.StringIO(uploaded["francais-fongbe-revu_sans-doublons2.txt"].decode("utf-8")), names=['fr', 'fongbe'], encoding="utf-8", engine="python", delimiter=" : ")

#lines= pd.read_table(uploaded[file_name], names=['fr', 'fongbe'], encoding="utf-8", engine="python", delimiter=" : ")



lines = lines[:30000];

# Lowercase all characters
lines.fr=lines.fr.apply(lambda x: x.lower())
lines.fongbe=lines.fongbe.apply(lambda x: x.lower())



# Remove quotes
lines.fr=lines.fr.apply(lambda x: re.sub("'", '', x))
lines.fongbe=lines.fongbe.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
#ponctuations = string.punctuation
#enlever = "'`-"
#exclude = ''.join(ch for ch in ponctuations if ch not in enlever)

# Remove all the special characters
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fongbe=lines.fongbe.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))


# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
#lines.fr=lines.fr.apply(lambda x: x.translate(remove_digits))
#lines.fongbe = lines.fongbe.apply(lambda x: x.translate(remove_digits))

#lines.fongbe = lines.fongbe.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines.fr=lines.fr.apply(lambda x: x.strip())
lines.fongbe=lines.fongbe.apply(lambda x: x.strip())
lines.fr=lines.fr.apply(lambda x: re.sub(" +", " ", x))
lines.fongbe=lines.fongbe.apply(lambda x: re.sub(" +", " ", x))


# Add start and end tokens to target sequences
lines.fongbe = lines.fongbe.apply(lambda x : 'START_ '+ x + ' _END')


# Vocabulary of English
all_fr_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_fr_words:
            all_fr_words.add(word)

# Vocabulary of French 
all_fongbe_words=set()
for fongbe in lines.fongbe:
    for word in fongbe.split():
        if word not in all_fongbe_words:
            all_fongbe_words.add(word)
            



# Max Length of source sequence
lenght_list=[]
for l in lines.fr:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)

# Max Length of target sequence
lenght_list=[]
for l in lines.fongbe:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)


input_words = sorted(list(all_fr_words))
target_words = sorted(list(all_fongbe_words))
num_encoder_tokens = len(all_fr_words)
num_decoder_tokens = len(all_fongbe_words)


print("fr vocab_size : ", len(all_fr_words))
print("fongbe vocab_size : ", len(all_fongbe_words))
#
num_decoder_tokens += 1 # For zero padding
num_encoder_tokens+= 1

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])


reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

latent_dim = 256

# creation du dictionnaire des données importantes à enregistrer pour utiliser plus tard

main_parameters = {
            
        "num_decoder_tokens" : num_decoder_tokens,
        "num_encoder_tokens" : num_encoder_tokens,
        "input_token_index" : input_token_index,
        "target_token_index" : target_token_index,
        "reverse_input_char_index": reverse_input_char_index,
        "reverse_target_char_index" : reverse_target_char_index,
        "latent_dim" : latent_dim,
        "max_length_tar" : max_length_tar,
        "max_length_src" : max_length_src,
        "input_words" : input_words,
        "target_words" : target_words
        }

with  open("main_parameters", 'wb') as file_:
    pickle.dump(main_parameters, file_)

    

# Train - Test Split
X, y = lines.fr, lines.fongbe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)



def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)


#Encoder - Decoder Model Architecture¶
    
    

#initializer of weight

weight_initializer = initializers.random_normal()
# Encoder
#encoder_inputs = Input(shape=(None,))
encoder_inputs = Input(shape=(None, ))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, recurrent_dropout=0.9)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)


# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, recurrent_dropout=0.9)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)


decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=None)

#optimizer =  SGD(lr=0.01 ,momentum=0.9 ,decay=1e-6, nesterov=True)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.summary()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs =80

#filepath="poids/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

#filepath='poids/model_train_weight.h5'
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',period=2)

es = EarlyStopping(monitor='val_loss', mode='min',patience=5,restore_best_weights=True)
callbacks_list = [es]

history = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size,
                    callbacks=callbacks_list
                  )

# evaluate the model
#_, train_acc = model.evaluate(trainX, trainy, verbose=0)
#_, test_acc = model.evaluate(testX, testy, verbose=0)
#print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

#plotting

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model.save_weights('nmt_weights.h5')

#saving du model
model.save("model_architecture.model")


#Load the weights, if you close the application
#model.load_weights('nmt_weights.h5')

#Inference Setup

# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Enregistrement des models dans des fichiers séparés
with open('encoder_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('encoder_model_weights.h5')

with open('decoder_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('decoder_model_weights.h5')


#Decode sample sequeces
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
#        if (sampled_char == '_END' or
#           len(decoded_sentence) > 50):
        
        if sampled_char == '_END' :
          
            stop_condition = True

                    

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence



#Evaluation on Train Dataset¶
#print("Evaluation on Train Dataset")

train_gen = generate_batch(X_train, y_train, batch_size = 1)
#k=-1

#k+=1

fichier_result = "traduction.txt"

fichier_resultat = open(fichier_result,"w",encoding="utf-8")


fichier_resultat.write("---------------- Traduction on Train Dataset -------------------\n")

for k in range(0,150): 
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Phrase Française entrée :', X_train[k:k+1].values[0])
    fichier_resultat.write('Phrase Française entrée :'+ X_train[k:k+1].values[0]+"\n")
    print('Phrase Fongbe normale :', y_train[k:k+1].values[0][6:-4])
    
    fichier_resultat.write('Phrase Fongbe normale :'+y_train[k:k+1].values[0][6:-4]+"\n")
    
    print('Phrase Fongbe traduite :', decoded_sentence[:-4])
    
    fichier_resultat.write('Phrase Fongbe traduite :'+ decoded_sentence[:-4]+"\n\n\n")
    
    

#Evaluation on Validation Dataset
print("Evaluation on Validation Dataset")

fichier_resultat.write("---------------- Traduction on Test Dataset -------------------\n\n\n")

val_gen = generate_batch(X_test, y_test, batch_size = 1)

for k in range(0,50):
    (input_seq, actual_output), _ = next(val_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Phrase Française entrée:', X_test[k:k+1].values[0])
    
    fichier_resultat.write('Phrase Française entrée:'+ X_test[k:k+1].values[0]+"\n")
    print('Phrase Fongbe normale:', y_test[k:k+1].values[0][6:-4])
    
    fichier_resultat.write('Phrase Fongbe normale:'+ y_test[k:k+1].values[0][6:-4]+"\n")
    
    print('Phrase Fongbe traduite:', decoded_sentence[:-4])
    
    fichier_resultat.write('Phrase Fongbe traduite:'+decoded_sentence[:-4]+"\n\n\n")
   



  




fr vocab_size :  10666
fongbe vocab_size :  12277
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_45 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, None, 256)    2730752     input_44[0][0]                   
__________________________________________________________________________________________________
embedding_25 (Embedding)        (None, None, 256)    3143168     input_45[0][0]                   
___________________________________________________________

In [14]:
from google.colab import files

list_fichiers = [
    "decoder_model.json",
    "encoder_model.json",
    "decoder_model_weights.h5",
    "encoder_model_weights.h5",
    "main_parameters",
    "model_architecture.model",
    "nmt_weights.h5",
    "traduction.txt"
]

files.download("decoder_model_weights.h5")
#for file in list_fichiers:
  
  #files.download(file)

KeyboardInterrupt: ignored